In [ ]:
# Importation des outils -------------------------------------------------------------------------------------------
import os 
import ipywidgets as widgets
from IPython.display import display
#-------------------------------------------------------------------------------------------------------------------

# Initialisation de GStreamer --------------------------------------------------------------------------------------
# On importe gi et on initialise GStreamer-->creer des pipelines.
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst
Gst.init(None)  # Pour démarrer GSt
#--------------------------------------------------------------------------------------------------------------------

# Pipelines de base -----------------------------------------------------------------------------
# On stocke nos pipelines audio, la vidéo et infos --> variables globales.
pipelineAUDIO = None   # lecture audio
pipelineVIDEO = None   # visualisation/affichage de la pochette
pipelineINFOS = None   # infos (dans une autre fenêtre)
#------------------------------------------------------------------------------------------------------------------------

# Récupération des fichiers dans le rep courant -----------------------------------------------------------------------
# On cherche les fichiers mp3, wav, flac seulement
DOSSIER_AUDIO = "./"
listeFichiersAudio = [f for f in os.listdir(DOSSIER_AUDIO) if f.endswith(('.mp3', '.wav', '.flac'))]
#-----------------------------------------------------------------------------------------------------------------------------

# Création du widget pour la sélection (avec ipywidgets) ------------------------------------------------------------------------------------
# Ici, on crée un petit menu avec les fic audio qui s'affiche pour en séléctionner 1.
selectionFichier = widgets.Dropdown(#RadioButtons rendait bien aussi mais c'était moins opti s'il y a beaucoup de fic
    options=listeFichiersAudio,
    description=" Sélection:",
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)
#--------------------------------------------------------------------------------------------------------------------------

# Création des différentes fonctions -----------------------------------------------------------------------------------
# fonction pour arreter toutes les pipelines pour eviter les bugs------------------------------------------------------
def arreterPipelines():
    global pipelineAUDIO, pipelineVIDEO, pipelineINFOS
    # On boucle sur chacunes des pipelines et on les arrête pour repartir sur une bonne base.
    for pipe in (pipelineAUDIO, pipelineVIDEO, pipelineINFOS):
        if pipe:
            pipe.set_state(Gst.State.NULL)  # on arrête tout
    pipelineAUDIO = pipelineVIDEO = pipelineINFOS = None  # puis on reinitialise
#----------------------------------------------------------------------------------------------------------------------------

# Fonction main --------------------------------------------------------------
def executerPipeline(_):
    global pipelineAUDIO, pipelineVIDEO, pipelineINFOS
    arreterPipelines()  # On appelle la fonction pour l'éxécuter (pour éviter les bugs si qqc tourne déjà)

    # On récupère le fichier sélectionné (si rien n'est choisi on stoppe)
    fichierSelectionne = selectionFichier.value
    if not fichierSelectionne:
        print("X Aucun fichier sélectionné.")
        return

    # si c'est bon on va créer leschemins fichier audio/pochette/infos
    cheminFichier = os.path.join(DOSSIER_AUDIO, fichierSelectionne)
    baseNom = os.path.splitext(fichierSelectionne)[0]#0 pour le 1er element du tuple genre x si le tuple c'est ("x", ".mp3")

    cheminPochette = os.path.join(DOSSIER_AUDIO, baseNom + ".jpg")
    cheminInfos = os.path.join(DOSSIER_AUDIO, baseNom + ".srt")
    
    # On crée la 1ere pipeline pour l'audio
    chaineAudio = "" #vide de base puis on va la remplir
    if toggleLecture.value:  # Si le bouton "Lire" est activé-->on construit la chaîne (qui sera différente en fonction du type de fichier car ce sera pas les mêmes greffons)                           
        if fichierSelectionne.endswith(".mp3"):#si c'est un mp3
            chaineAudio = (f"filesrc location={cheminFichier} ! mpegaudioparse ! "
                           f"mpg123audiodec ! audioresample ! autoaudiosink")
        elif fichierSelectionne.endswith(".wav"):#si c'est un wav
            chaineAudio = f"filesrc location={cheminFichier} ! wavparse ! audioresample ! autoaudiosink"
        elif fichierSelectionne.endswith(".flac"):#si c'est un flac
            chaineAudio = f"filesrc location={cheminFichier} ! flacparse ! flacdec ! audioresample ! autoaudiosink"
    
    # Mtn, on crée la pipeline video avec visualisation/affichage de la pochette
    # On prépare la chaîne pour la visualisation et celle pour la pochette
    #si les options sont activées on les lance
    chaineVisual = (f"filesrc location={cheminFichier} ! decodebin ! audioconvert ! wavescope "#On a utilisé decodebin içi (même si vous avez dis qu'il fallait mieux éviter) car on avait des bugs sinon
                    f"! videoconvert ! videoscale ! video/x-raw,width=640,height=360") if toggleVisual.value else ""#x-raw,width...-->format de la video
    chainePochette = (f"filesrc location={cheminPochette} ! jpegdec ! imagefreeze ! videoconvert ! videoscale "
                      f"! video/x-raw,width=640,height=360") if togglePochette.value and os.path.exists(cheminPochette) else ""
    #la pipeline chainVisual sans decodebin qui posé problème : 
    """chaineVisual = (f"filesrc location={cheminFichier} ! qtdemux name=demux "
                f"demux.audio_0 ! queue ! mpegaudioparse ! mpg123audiodec ! audioconvert ! wavescope "
                f"! videoconvert ! videoscale ! video/x-raw,width=640,height=360")
                """
    
    # si les 2 sont activées ont les combine ds une seule fenêtre car sinon les fenetres vont s'entasser et l'affichage sera pas terrible
    chaineVideo = ""
    if chaineVisual and chainePochette:
        # Pour les mettre ds la même fenêtre, on utilise le greffon "compositor".
        chaineVideo = (f"compositor name=mix ! videoconvert ! autovideosink "
                       f"({chaineVisual} ! queue ! mix.sink_0) "
                       f"({chainePochette} ! queue ! mix.sink_1)")
    elif chaineVisual: # si il n'y a que la visu d'activée comme option
        chaineVideo = chaineVisual + " ! autovideosink"
    elif chainePochette:# si il n'y a que la pochette d'activée comme option
        chaineVideo = chainePochette + " ! autovideosink"
    
    # On lance les 2 pipelines qu'on a créé
    if chaineAudio:# Si on a bien construit la pipeline audio-->on la lance.
        print("Pipeline audio:", chaineAudio)
        pipelineAUDIO = Gst.parse_launch(chaineAudio)
        pipelineAUDIO.set_state(Gst.State.PLAYING)#passe la pipeline en état "playing" (en gros cette ligne la lance pour que l'audio soit lu immédiatement)
    if chaineVideo:#pareil pour la pipeline video
        print("Pipeline vidéo:", chaineVideo)
        pipelineVIDEO = Gst.parse_launch(chaineVideo)
        pipelineVIDEO.set_state(Gst.State.PLAYING)
    
    # On crée une 3eme pipeline pour les infos du fichier qu'on va lancer dans une 2eme fenetre à part
    if toggleInfos.value and os.path.exists(cheminInfos):
        with open(cheminInfos, "r", encoding="utf-8") as f:
            lignes = [ligne.strip() for ligne in f.readlines() if ligne.strip()]  # Supprime les lignes vides

        #print("Méta-données extraites :", lignes)  #Pour voir si les infos sont bien récup

        texteInfos = []#initialisation
        for ligne in lignes:
            if "Interprete" in ligne:
                texteInfos.append(f"\nInterprète : {ligne.split(':')[-1].strip()}")
                #split pour séparer en fonction des ":"
                #strip pour enlever les espaces inutiles
                #\n pour passer à la ligne suivante (mettre en colonne)
            elif "Titre" in ligne:
                texteInfos.append(f"\nTitre : {ligne.split(':')[-1].strip()}")
            elif "Album" in ligne:
                texteInfos.append(f"\nAlbum : {ligne.split(':')[-1].strip()}")

        texteInfos = "\\n".join(texteInfos)  #Reformate en 1 chaîne

        print("Texte final envoyé à textoverlay :", texteInfos)  #Pour voir ce qui est envoyé


        chaineInfos = (f'videotestsrc pattern=black ! videoconvert '
               f'! textoverlay text="{texteInfos}" valignment=top halignment=center valignment=center font-desc="Sans 18, bold" '
               f'! autovideosink') #permet de créer la 2eme fenètre en mettant les infos sur fond noir
        print("Pipeline infos:", chaineInfos)
        pipelineINFOS = Gst.parse_launch(chaineInfos)#on la lance si l'option est utilisée
        pipelineINFOS.set_state(Gst.State.PLAYING)

# La fonction pour mettre pause dans le son
def basculerPause(_):
    global pipelineAUDIO, pipelineVIDEO
    # On récupère tt les pipelines qui tournent pour appliquer la commande partout.
    pipelines = [p for p in (pipelineAUDIO, pipelineVIDEO) if p]
    if not pipelines:
        return
    etat = pipelines[0].get_state(0).state  #On vérifie l'état du 1er pipeline (souvent ça suffit pour savoir quoi faire)
    if etat == Gst.State.PLAYING:
        for p in pipelines:
            p.set_state(Gst.State.PAUSED)  # On met tt en pause
        boutonPauseReprise.description = " Reprendre"  # On change le texte du bouton en "Reprendre" (ça marche pas à chaque fois)
    else:
        for p in pipelines:
            p.set_state(Gst.State.PLAYING)  # On relance la lecture
        boutonPauseReprise.description = "Pause"

# Fonction pour avancer ou reculer (la lecture du son)
def deplacerPosition(offset):
    if not pipelineAUDIO:
        return
    success, posActuelle = pipelineAUDIO.query_position(Gst.Format.TIME)
    if not success:
        print("Impossible d'obtenir la position.")
        return
    nouvellePos = max(0, posActuelle + offset * Gst.SECOND)  # On s'assure de ne pas avoir de position impossible (par exemple -2sec/1Min30)
    pipelineAUDIO.seek_simple(Gst.Format.TIME, Gst.SeekFlags.FLUSH | Gst.SeekFlags.KEY_UNIT, nouvellePos)
    #FLUSH pour vider le buffer en attente --> que des données récentes
    #KEY_UNIT pour se reppositionner sur une "key unit" --> pas une demi-frame par exemple


def avancer(_):
    deplacerPosition(10)#positif = avancer

def reculer(_):
    deplacerPosition(-10)#négatif = reculer

"""Test pour créer une barre de volume mais la gestion du son ne marchait pas

import time

def changerVolume(change):
    global pipelineAUDIO
    if pipelineAUDIO:
        
        time.sleep(0.5)#pour être sûr que ce soit bien initialisé
        
        volume_element = pipelineAUDIO.get_by_name("controle_volume")  #Récup le bon élément
        if volume_element:
            volume_element.set_property("volume", change["new"])  #Applique le volume
            print(f" Volume modifié : {change['new']}")  # log
        else:
            print(" Erreur : Élément de volume introuvable ")#log d'erreur
            sliderVolume.observe(changerVolume, names='value')
"""
#----------------------------------------------------------------------------------------------------------------------------

# Création des boutons ---------------------------------------------------------------------------------------------
# Avec ces boutons , on peut activer/désactiver la lecture (qu'on met à activé de base), la visualisation, la pochette et l'affichage des infos.
toggleLecture  = widgets.ToggleButton(value=True,  description=" Lire",          button_style="primary")#True pour que ce soit actif de base 
toggleVisual   = widgets.ToggleButton(value=False, description=" Visualisation", button_style="info")
togglePochette = widgets.ToggleButton(value=False, description=" Pochette",      button_style="warning")
toggleInfos    = widgets.ToggleButton(value=False, description=" Infos")

boutonExecution = widgets.Button(description=" Exécuter", button_style="success")
#Ces trois autres boutons permettent de mettre en pause/avancer/reculer la musique si la lecture de la musique a été activé
boutonPauseReprise = widgets.Button(description=" || Pause", button_style="danger")

boutonAvancer      = widgets.Button(description=" Avancer-->", button_style="primary")
boutonAvancer.style.button_color = 'black'
boutonReculer      = widgets.Button(description=" <--Reculer", button_style="primary")
boutonReculer.style.button_color = 'black'
#----------------------------------------------------------------------------------------------------------------------------

#Liaisons bouton/fonction---------------------------------------------------------------------------------------------------
boutonPauseReprise.on_click(basculerPause)
boutonAvancer.on_click(avancer)
boutonReculer.on_click(reculer)
boutonExecution.on_click(executerPipeline)
#----------------------------------------------------------------------------------------------------------------------------------

# On affiche notre interface avec tous nos boutons bien alignés----------------------------------------------------------
layout=widgets.Layout(
    align_items='center',#centre le contenu
    padding='20px',#espacement entre les boutons
    border='solid 2px black',#contour noir
    background_color='#f0f0f0',#Fond gris clair normalement mais ça marche pas
    width='100%'  #la largeur
)
zoneControles = widgets.HBox([boutonReculer, boutonPauseReprise, boutonAvancer])#Hbox=boîte horizontale donc ça les place en ligne et pas en colonne

boiteInterface = widgets.VBox([#L'inverse
    selectionFichier,
    toggleLecture, toggleVisual, togglePochette, toggleInfos,
    boutonExecution,
    zoneControles
], layout=layout)

display(boiteInterface)#pour afficher
#-----------------------------------------------------------------------------------------------------------------------------


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Pipeline audio: filesrc location=./PF_short.wav ! wavparse ! audioresample ! autoaudiosink
Pipeline vidéo: compositor name=mix ! videoconvert ! autovideosink (filesrc location=./PF_short.wav ! decodebin ! audioconvert ! wavescope ! videoconvert ! videoscale ! video/x-raw,width=640,height=360 ! queue ! mix.sink_0) (filesrc location=./PF_short.jpg ! jpegdec ! imagefreeze ! videoconvert ! videoscale ! video/x-raw,width=640,height=360 ! queue ! mix.sink_1)
Pipeline audio: filesrc location=./PF_short.wav ! wavparse ! audioresample ! autoaudiosink
Pipeline vidéo: filesrc location=./PF_short.jpg ! jpegdec ! imagefreeze ! videoconvert ! videoscale ! video/x-raw,width=640,height=360 ! autovideosink
Pipeline audio: filesrc location=./PF_short.wav ! wavparse ! audioresample ! autoaudiosink
Pipeline vidéo: filesrc location=./PF_short.wav ! decodebin ! audioconvert ! wavescope ! videoconvert ! videoscale ! video/x-raw,width=640,height=360 ! autovideosink
Pipeline audio: filesrc location=./PF_short.